In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
def compare(calc1, calc2, syr, eyr):
    """
    Compare aggregate tax liability under two calculators
    where calc1 is the baseline and calc2 is the new/reform
    """
    # dictionary to hold aggregate liability
    tax_liability = {'year': [], 'base': [], 'new': [], 'diff': []}

    # loop through each year collecting aggregate tax liabilities
    for year in range(syr, eyr + 1):
        try:
            calc1.advance_to_year(year)
            calc1.calc_all()
            tax1 = calc1.weighted_total('combined')
        except ValueError:
            tax1 = np.nan
        calc2.advance_to_year(year)
        calc2.calc_all()
        tax2 = calc2.weighted_total('combined')
        diff = tax2 - tax1
        tax_liability['year'].append(year)
        tax_liability['base'].append(tax1)
        tax_liability['new'].append(tax2)
        tax_liability['diff'].append(diff)
    return tax_liability

# PUF

In [3]:
puf_path = 'puf_data/puf.csv'
gfactors_path = '/puf_stage1/growfactors.csv'
gfactors = pd.read_csv('puf_stage1/growfactors.csv', index_col='YEAR')
puf_weights = pd.read_csv('puf_stage2/puf_weights.csv.gz')
puf_ratios = pd.read_csv('puf_stage3/puf_ratios.csv', index_col=0).transpose().reset_index(drop=True, inplace=True)
puf_ratios_path = 'puf_stage3/puf_ratios.csv'

In [4]:
calc_base = tc.Calculator(records=tc.Records(puf_path), policy=tc.Policy())
gfactors_new = tc.GrowFactors(gfactors_path)
setattr(gfactors_new, 'gfdf', gfactors)
recs_new = tc.Records(puf_path, gfactors=gfactors_new,
                      weights=puf_weights,
                      adjust_ratios=puf_ratios)
pol_new = tc.Policy(gfactors=gfactors_new)
puf_behavior = tc.Behavior(num_years=16)
calc_new = tc.Calculator(records=recs_new, policy=pol_new,
                         behavior=puf_behavior)

FUTURE: use the Behavioral-Responses behresp package OR
        use the Tax-Calculator quantity_response function.
You loaded data for 2011.
Your data include the following unused variables that will be ignored:
  filer
Tax-Calculator startup automatically extrapolated your data to 2013.
FUTURE: use the Behavioral-Responses behresp package OR
        use the Tax-Calculator quantity_response function.
You loaded data for 2011.
Your data include the following unused variables that will be ignored:
  filer
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
puf_comps = compare(calc_base, calc_new, 2018, 2027)

In [6]:
puf_comp_df = pd.DataFrame(puf_comps)
puf_comp_df['% diff'] = (puf_comp_df['diff'] / puf_comp_df['base']) * 100
puf_comp_df.style.format("${:,.2f}", subset=['base', 'new', 'diff'])

,year,base,new,diff,% diff
0,2018,"$2,697,635,861,190.10","$2,656,835,033,057.69","$-40,800,828,132.40",-1.51247
1,2019,"$2,806,712,706,848.11","$2,836,125,801,659.90","$29,413,094,811.79",1.04796
2,2020,"$2,913,652,711,942.36","$2,966,825,970,774.47","$53,173,258,832.10",1.82497
3,2021,"$3,030,799,622,984.32","$3,087,463,637,684.19","$56,664,014,699.86",1.86961
4,2022,"$3,160,565,814,412.93","$3,212,674,487,345.93","$52,108,672,933.00",1.64871
5,2023,"$3,302,696,281,709.51","$3,348,396,610,472.89","$45,700,328,763.37",1.38373
6,2024,"$3,451,801,384,758.67","$3,495,307,011,503.59","$43,505,626,744.92",1.26037
7,2025,"$3,607,438,166,542.78","$3,650,936,974,017.17","$43,498,807,474.39",1.20581
8,2026,"$4,003,922,840,844.45","$4,047,121,028,092.82","$43,198,187,248.38",1.0789
9,2027,"$4,184,166,034,209.22","$4,227,835,462,363.94","$43,669,428,154.72",1.04368


# CPS

In [7]:
cps_path = 'cps_data/cps.csv.gz'
cps_weights = pd.read_csv('cps_stage2/cps_weights.csv.gz')
gfactors_cps = tc.GrowFactors(gfactors_path)

In [8]:
cps_base = tc.Calculator(records=tc.Records.cps_constructor(), policy=tc.Policy())
cps_recs_new = tc.Records(cps_path, gfactors=gfactors_new,
                          weights=cps_weights,
                          adjust_ratios=None,
                          start_year=2014)
cps_pol_new = tc.Policy(gfactors=gfactors_new)
cps_behavior = tc.Behavior(num_years=16)
cps_new = tc.Calculator(records=cps_recs_new, policy=cps_pol_new,
                        behavior=cps_behavior)

FUTURE: use the Behavioral-Responses behresp package OR
        use the Tax-Calculator quantity_response function.
You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  filer
Tax-Calculator startup automatically extrapolated your data to 2014.
FUTURE: use the Behavioral-Responses behresp package OR
        use the Tax-Calculator quantity_response function.
You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  filer
Tax-Calculator startup automatically extrapolated your data to 2014.


In [9]:
cps_comps = compare(cps_base, cps_new, 2018, 2027)

In [10]:
cps_comps_df = pd.DataFrame(cps_comps)
cps_comps_df['% diff'] = (cps_comps_df['diff'] / cps_comps_df['base']) * 100
cps_comps_df.style.format("${:,.2f}", subset=['base', 'new', 'diff'])

,year,base,new,diff,% diff
0,2018,"$2,487,143,115,305.13","$2,425,285,433,242.08","$-61,857,682,063.04",-2.4871
1,2019,"$2,611,372,571,132.18","$2,605,731,240,722.92","$-5,641,330,409.25",-0.216029
2,2020,"$2,729,622,113,054.46","$2,763,788,243,925.94","$34,166,130,871.48",1.25168
3,2021,"$2,856,651,208,216.11","$2,907,535,885,601.39","$50,884,677,385.29",1.78127
4,2022,"$2,991,674,907,566.06","$3,047,566,327,369.07","$55,891,419,803.01",1.86823
5,2023,"$3,132,830,824,329.71","$3,189,183,768,467.96","$56,352,944,138.25",1.79879
6,2024,"$3,276,581,707,266.39","$3,333,665,296,079.49","$57,083,588,813.10",1.74217
7,2025,"$3,419,847,598,269.39","$3,478,388,591,538.76","$58,540,993,269.38",1.7118
8,2026,"$3,775,395,519,908.75","$3,835,085,165,096.18","$59,689,645,187.43",1.58102
9,2027,"$3,938,500,239,015.11","$3,998,984,962,583.33","$60,484,723,568.22",1.53573
